In [10]:
import rawpy
import numpy as np
import cv2
import pandas as pd

In [11]:
from find_stars import StarFinder

In [12]:
out_of_focus = r"D:\Astro\20220430-whale\outoffocus\Image741.nef"
in_focus = r"D:\Astro\20220430-whale\light\Image752.nef"

s = StarFinder()

a_stars = s.getStarData(in_focus)
b_stars = s.getStarData(out_of_focus)


In [13]:
def getStarDF(stardata):
  df = pd.DataFrame(stardata)
  df.drop(0, axis=0, inplace=True)
  return df


In [14]:
getStarDF(a_stars)

,area,centroid,box
1,43,"[560.8372093023256, 1.9069767441860466]","[551, 0, 570, 7]"
2,88,"[2968.9204545454545, 180.60227272727272]","[2959, 170, 2978, 191]"
3,70,"[2920.4, 182.1]","[2912, 173, 2929, 192]"
4,206,"[2233.9854368932038, 241.9126213592233]","[2218, 224, 2249, 259]"
5,49,"[679.0, 495.0]","[672, 488, 687, 503]"
6,151,"[1154.4370860927152, 579.5165562913908]","[1142, 565, 1167, 594]"
7,49,"[2172.0, 793.0]","[2165, 786, 2180, 801]"
8,56,"[1510.0, 864.5]","[1503, 856, 1518, 873]"
9,177,"[1913.9943502824858, 933.6836158192091]","[1900, 918, 1927, 949]"
10,63,"[2265.5555555555557, 951.4444444444445]","[2257, 943, 2274, 960]"


In [15]:
getStarDF(b_stars)

,area,centroid,box
1,56,"[749.5, 35.0]","[741, 28, 758, 43]"
2,121,"[2658.7024793388427, 247.25619834710744]","[2647, 235, 2670, 260]"
3,64,"[2609.5, 250.5]","[2601, 242, 2618, 259]"
4,151,"[4260.503311258278, 297.28476821192055]","[4247, 284, 4274, 311]"
5,300,"[1923.49, 308.10333333333335]","[1905, 288, 1942, 329]"
6,56,"[1733.5, 328.0]","[1725, 321, 1742, 336]"
7,49,"[3803.0, 349.0]","[3796, 342, 3811, 357]"
8,112,"[4181.321428571428, 577.3303571428571]","[4170, 566, 4193, 589]"
9,230,"[844.9, 645.4739130434782]","[828, 628, 861, 663]"
10,49,"[782.0, 701.0]","[775, 694, 790, 709]"
